# Autotagging

In [1]:
import torch
import torchaudio
import IPython.display as ipd
from pathlib import Path
import pandas as pd

- Download Small MTAT if you need


In [3]:
!pip install --upgrade gdown
!gdown 15e9E3oZdudErkPKwb0rCAiZXkPxdZkV6
!unzip -q mtat_8000.zip

  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Downloading...
From (uriginal): https://drive.google.com/uc?id=15e9E3oZdudErkPKwb0rCAiZXkPxdZkV6
From (redirected): https://drive.google.com/uc?id=15e9E3oZdudErkPKwb0rCAiZXkPxdZkV6&confirm=t&uuid=82f01663-4f06-4fdd-8b20-5fbc61f7609f
To: /home/teo/userdata/git_libraries/ant5015-2023/notebooks/mtat_8000.zip
100%|████████████████████████████████████████| 921M/921M [01:53<00:00, 8.15MB/s]


In [6]:
class MTATDataset:
  def __init__(self, dir_path, split='train', num_max_data=6000, sr=16000):
    self.dir = Path(dir_path)
    self.labels = pd.read_csv(self.dir / "meta.csv", index_col=[0])
    self.sr = sr

    if split=="train":
      sub_dir_ids = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c']
    elif split=='valid':
      sub_dir_ids = ['d']
    else: #test
      sub_dir_ids = ['e', 'f', 'g']

    is_in_set = [True if x[0] in sub_dir_ids else False for x in self.labels['mp3_path'].values.astype('str')]
    self.labels = self.labels.iloc[is_in_set]
    self.labels = self.labels[:num_max_data]
    self.vocab = self.labels.columns.values[1:-1]
    self.label_tensor = self.convert_label_to_tensor()
  
  def convert_label_to_tensor(self):
    return torch.LongTensor(self.labels.values[:, 1:-1].astype('bool'))

  def __len__(self):
    return len(self.labels)

data_dir = Path('MTAT_SMALL')
dataset= MTATDataset(data_dir)

## Make Dataset

## Make Model

## Train!

## Data Normalization

## How CNN works

In [12]:
dummy = torch.randint(-3, 4, (2, 6,7)).float()
dummy = dummy.unsqueeze(0)
dummy, dummy.shape

(tensor([[[[ 2.,  3.,  0., -2., -2.,  1., -2.],
           [-3., -3.,  1.,  2., -3.,  1.,  3.],
           [-2.,  1.,  0., -3.,  0., -1., -2.],
           [ 0., -3., -3., -2., -2., -3.,  1.],
           [ 2.,  3.,  2.,  3.,  3., -2., -2.],
           [ 1., -2., -1., -3., -1., -1.,  0.]],
 
          [[ 0.,  0.,  3.,  3., -3., -3.,  0.],
           [-2.,  0.,  2.,  3.,  2., -3., -3.],
           [-1.,  1.,  0., -2.,  3.,  1.,  3.],
           [ 1.,  3.,  0.,  0.,  2.,  3.,  3.],
           [-3.,  2., -3., -3.,  3., -1.,  2.],
           [-3.,  0.,  1., -1.,  1.,  3.,  0.]]]]),
 torch.Size([1, 2, 6, 7]))

In [13]:
import torch.nn as nn
conv_layer = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=3, bias=False)
conv_layer.weight

Parameter containing:
tensor([[[[ 0.0329,  0.0913, -0.0989],
          [-0.1222, -0.0248,  0.1674],
          [ 0.0012,  0.0244, -0.2151]],

         [[-0.1140,  0.2020,  0.1060],
          [ 0.0658, -0.0340, -0.0514],
          [-0.0025, -0.1469,  0.1981]]]], requires_grad=True)

In [15]:
conv_layer.weight.shape # out_channels, in_channels, height, width
conv_layer.weight.data = torch.randint(-1, 2, (1,2,3,3)).float()
conv_layer.weight

Parameter containing:
tensor([[[[ 1.,  1., -1.],
          [ 0., -1., -1.],
          [ 0.,  1.,  0.]],

         [[ 1.,  0.,  0.],
          [-1.,  0.,  1.],
          [ 1.,  1.,  0.]]]], requires_grad=True)

In [18]:
'''
2D Conv Layer의 입력은 3차원

'''
dummy, conv_layer.weight, conv_layer(dummy)

(tensor([[[[ 2.,  3.,  0., -2., -2.,  1., -2.],
           [-3., -3.,  1.,  2., -3.,  1.,  3.],
           [-2.,  1.,  0., -3.,  0., -1., -2.],
           [ 0., -3., -3., -2., -2., -3.,  1.],
           [ 2.,  3.,  2.,  3.,  3., -2., -2.],
           [ 1., -2., -1., -3., -1., -1.,  0.]],
 
          [[ 0.,  0.,  3.,  3., -3., -3.,  0.],
           [-2.,  0.,  2.,  3.,  2., -3., -3.],
           [-1.,  1.,  0., -2.,  3.,  1.,  3.],
           [ 1.,  3.,  0.,  0.,  2.,  3.,  3.],
           [-3.,  2., -3., -3.,  3., -1.,  2.],
           [-3.,  0.,  1., -1.,  1.,  3.,  0.]]]]),
 Parameter containing:
 tensor([[[[ 1.,  1., -1.],
           [ 0., -1., -1.],
           [ 0.,  1.,  0.]],
 
          [[ 1.,  0.,  0.],
           [-1.,  0.,  1.],
           [ 1.,  1.,  0.]]]], requires_grad=True),
 tensor([[[[ 12.,   6.,  -1.,  -5.,  -8.],
           [ -8.,  -4.,  12.,   5.,   2.],
           [  5.,   8.,   0.,   7.,   7.],
           [ -9., -11.,  -6.,  -1.,   2.]]]], grad_fn=<ThnnConv2DBackw

In [26]:
conv_layer(dummy[:, 0:3, 1:4])

tensor([[[0.]]], grad_fn=<SqueezeBackward1>)

In [41]:
num_ch = 4
dummy = torch.randint(-3, 4, (num_ch, 10)).float()
dummy

tensor([[ 2., -3.,  0., -3.,  0.,  2.,  2., -1.,  3.,  1.],
        [-2., -1.,  3., -3.,  2.,  0.,  0.,  2.,  3., -3.],
        [ 1., -2., -3.,  0.,  1.,  3., -3.,  1.,  3., -2.],
        [-1., -2., -2., -3.,  0., -3., -3., -1.,  1.,  3.]])

In [46]:
out_ch = 6
conv1d = nn.Conv1d(num_ch, out_ch, kernel_size=3, bias=False)
conv1d.weight.data = torch.randint(-1, 2, conv1d.weight.shape).float()
conv1d.weight

Parameter containing:
tensor([[[-1.,  1., -1.],
         [ 0.,  0., -1.],
         [ 1.,  0.,  1.],
         [ 0.,  0.,  0.]],

        [[-1.,  1., -1.],
         [-1.,  1., -1.],
         [ 0.,  1.,  0.],
         [ 1., -1.,  1.]],

        [[ 1., -1., -1.],
         [ 1.,  1.,  1.],
         [-1.,  1., -1.],
         [ 0.,  1.,  1.]],

        [[-1.,  1.,  1.],
         [-1., -1., -1.],
         [-1., -1., -1.],
         [ 1.,  1.,  1.]],

        [[ 1.,  1.,  1.],
         [-1.,  0.,  0.],
         [ 0.,  1.,  1.],
         [ 1., -1.,  1.]],

        [[ 0.,  1., -1.],
         [ 0.,  0., -1.],
         [ 0.,  0., -1.],
         [-1., -1.,  1.]]], requires_grad=True)

In [47]:
out = conv1d(dummy)
out, out.shape

(tensor([[-10.,   7.,  -7.,   4.,  -2.,   3.,  -9.,   5.],
         [-10.,   7., -10.,   1.,   1.,  -5.,  -7.,  11.],
         [  1.,  -7.,   4., -11.,  -3.,  -8.,   6.,   5.],
         [ -6.,  -1.,  -8.,  -4.,  -5., -11.,  -9.,   4.],
         [ -5., -11.,  -4.,   0.,   2.,   0.,   7.,   3.],
         [ -2.,   7.,  -1.,  -5.,   3.,   5.,  -5.,  10.]],
        grad_fn=<SqueezeBackward1>),
 torch.Size([6, 8]))

In [52]:
max_pool_layer = nn.MaxPool1d(3)

In [53]:
after_pool = max_pool_layer(out)
after_pool, after_pool.shape

(tensor([[ 7.,  4.],
         [ 7.,  1.],
         [ 4., -3.],
         [-1., -4.],
         [-4.,  2.],
         [ 7.,  5.]], grad_fn=<SqueezeBackward1>),
 torch.Size([6, 2]))

## Batch Norm

In [187]:
audio

tensor([[     0.0000,      0.0000,      0.0000,  ...,      0.0000,
             -0.0000,     -0.0000],
        [     0.0000,      0.0000,      0.0000,  ...,     -0.0000,
             -0.0000,     -0.0000],
        [     0.0000,      0.0000,      0.0000,  ...,     -0.0000,
              0.0000,      0.0000],
        ...,
        [     0.0000,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        [     0.0000,      0.0000,      0.0000,  ...,     -0.0000,
             -0.0000,     -0.0000],
        [     0.0000,      0.0000,      0.0000,  ...,     -0.0000,
             -0.0000,     -0.0000]])

In [ ]:
model.train()
model.eval()